In [29]:
import cohere 
from ratelimit import limits, sleep_and_retry

co = cohere.Client(
  api_key="", # This is your trial API key
) 

# Define rate limit: 10 calls per 60 seconds
CALLS = 5
PERIOD = 60

@sleep_and_retry
@limits(calls=CALLS, period=PERIOD)
def eval(row):
  inputprompt = row["inputs"]
  answer = row["targets"]

  response = co.chat( 
    model='c4ai-aya-23-8b',
    message=f'''[System]
  Please act as an impartial judge and evaluate the accuracy of the response provided by an
  AI assistant to the user input displayed below. Please rate your confidence on a scale of 1 to 100
  by strictly following this format: "[[confidence]]", for example: "Confidence: [[95]]". After doing this, provide a short explanation, citing specific examples in the input to justify your score.
  [Input Prompt]
  Identify the language of the following: {inputprompt}
  [The Start of Assistant’s Answer]
  {answer}
  [The End of Assistant’s Answer]''',
  ) 
  return response.text

In [30]:
import pandas as pd

df = pd.read_csv("hf://datasets/mislabel-indentification-aya/aya_dataset_mislabeled/mislabeled_samples_CohereForAI_aya_dataset_train.csv")

In [31]:
df.head()

,index,inputs,targets,language,language_code,annotation_type,user_id,inputs_lid,targets_lid,category,comment
0,1912,Create a dialogue between a parent and a teena...,माता-पिता और एक किशोर के बीच एक संवाद बनाएं जि...,Hindi,hin,original-annotations,b4597c1584ff2fbf47d6e6cf59fc965eb13c43edbdae05...,eng,hin,instruction_not_followed,hindi_translation_of_english_input
1,4224,Create a dialogue between two friends discussi...,दो दोस्तों के बीच एक संवाद बनाएं जिनमें समाज प...,Hindi,hin,original-annotations,b4597c1584ff2fbf47d6e6cf59fc965eb13c43edbdae05...,eng,hin,instruction_not_followed,hindi_translation_of_english_input
2,10070,Write a complex political thriller that unfold...,एक जटिल राजनीतिक थ्रिलर लिखें जो एक भविष्य में...,Hindi,hin,original-annotations,b4597c1584ff2fbf47d6e6cf59fc965eb13c43edbdae05...,eng,hin,instruction_not_followed,hindi_translation_of_english_input
3,12038,Write a dialogue-heavy screenplay centered aro...,एक महत्वपूर्ण मुद्दे पर विवादात्मक राजनीतिक बह...,Hindi,hin,original-annotations,b4597c1584ff2fbf47d6e6cf59fc965eb13c43edbdae05...,eng,hin,instruction_not_followed,hindi_translation_of_english_input
4,14733,Craft an epic tale set in a world where magic ...,मुझे लगता है आप कहना चाहते हैं एक ऐसी दुनिया म...,Hindi,hin,re-annotations,6a814b9adfea95c45ce3892528646106af167aad3bafa8...,eng,hin,instruction_not_followed,hindi_translation_of_english_input


In [32]:
df["llm_judge"] = df.apply(eval, axis=1)

ReadTimeout: The read operation timed out